In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME-EN.csv", sep=';')
election_data

In [ ]:
# Extract Barcelona data
barcelona_data = election_data[election_data['Municipality_name'] == 'Barcelona']
# Add up votes for each district of Barcelona
district_data = barcelona_data.iloc[:, 21:].groupby(by=barcelona_data["District"]).sum()
district_data.reset_index()

In [ ]:
# Find the winner party of every district
winner_by_district=district_data.idxmax(axis=1)
winner_by_district.reset_index()
list_winners=list(winner_by_district)
# Add column with winner party
district_data['winner']=list_winners
district_data

In [ ]:
# Read Barcelona's districts map
bcn = gpd.read_file(data_dir + "districts_BCN.geojson")
bcn.head()
# Select only the columns 'NOM' and 'geometry' from the GeoDataFrame
bcn = bcn[['NOM','geometry']]
# Rename the 'DISTRICTE' column to 'District'
bcn.rename(columns={'DISTRICTE': 'District'}, inplace=True)
# Create a new GeoDataFrame with the selected columns
bcn_gpd = gpd.GeoDataFrame(bcn)
# Add a new column 'District' with values 1 to 10
bcn_gpd['District'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
bcn_gpd

In [ ]:
# Merge Barcelona's districts map with votes in each district
merged_data = bcn_gpd.merge(district_data, on='District')
merged_data.to_csv('election_results_bcn.csv', index=False)


In [ ]:
# Plot the election results on a map of Barcelona's districts

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='winner',  # Change to the column you want to visualize
    hover_name='District',
    mapbox_style="carto-positron",
    color_discrete_sequence=["darkred", "goldenrod", "lightgreen", "royalblue", "black",'purple'],
    center={"lat": 41.3874, "lon": 2.1686},  # Center of Catalonia
    zoom=10,
    title='Election results by Barcelona district'
)

fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()